In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from scipy.sparse import csr_matrix
review_json_path = 'Books_5.json'

size = 1000000
review = pd.read_json(review_json_path, lines=True,
                      chunksize=size)


for chunk_review in review:
    sample_list = pd.DataFrame(chunk_review[['reviewerID','asin','overall']])
    
    print('run')


run


KeyboardInterrupt: 

In [2]:
book_count = (sample_list.
     groupby(by = ['asin'])['overall'].
     count().
     reset_index()
    )


threshold = 50
book_count = book_count.query('overall >= @threshold')

user_rating = pd.merge(book_count, sample_list, left_on='asin', right_on='asin', how='left')

user_count = (sample_list.
     groupby(by = ['reviewerID'])['overall'].
     count().
     reset_index()
    )

threshold = 20
user_count = user_count.query('overall >= @threshold')

combined = user_rating.merge(user_count, left_on = 'reviewerID', right_on = 'reviewerID', how = 'inner')

combined.head(10)

,asin,overall_x,reviewerID,overall_y,overall
0,000100039X,772,A2UIILZDC35P15,5,27
1,0001048767,3018,A2UIILZDC35P15,5,27
2,0001050230,2228,A2UIILZDC35P15,5,27
3,0007107005,249,A2UIILZDC35P15,4,27
4,0007155662,4575,A2UIILZDC35P15,3,27
5,0007250916,885,A2UIILZDC35P15,5,27
6,0007350899,7894,A2UIILZDC35P15,5,27
7,0007350961,1856,A2UIILZDC35P15,5,27
8,0007548672,15922,A2UIILZDC35P15,5,27
9,0008218439,521,A2UIILZDC35P15,5,27


In [3]:
combined_gb = combined.groupby(by = ['reviewerID','asin'],as_index=False).mean()

combined_final = combined_gb.drop(columns = ['overall_x','overall'])
combined_final = combined_final.rename(columns = {'reviewerID':'User-ID', 'asin': 'Book-ID', 'overall_y':'Rating'})
scaler = MinMaxScaler()
combined_final['Rating'] = combined_final['Rating'].values.astype(float)
rating_scaled = pd.DataFrame(scaler.fit_transform(combined_final['Rating'].values.reshape(-1,1)))
combined_final['Rating'] = rating_scaled



In [4]:
combined_final.head(10)

,User-ID,Book-ID,Rating
0,A100YDQ9DIP06P,0001047868,1.00
1,A100YDQ9DIP06P,0060219106,1.00
2,A100YDQ9DIP06P,0060232943,0.75
3,A100YDQ9DIP06P,0060766212,1.00
4,A100YDQ9DIP06P,0060783338,0.75
5,A100YDQ9DIP06P,0061174297,1.00
6,A100YDQ9DIP06P,0061284513,1.00
7,A100YDQ9DIP06P,0061671797,1.00
8,A102Z3T7NSM5KC,0002246791,0.50
9,A102Z3T7NSM5KC,0007149832,0.75


At this point, the data is in a scaled tabular format with user-ID, Book-ID, and Ratin

In [5]:
combined = combined_final.drop_duplicates(['User-ID', 'Book-ID'])
user_book_matrix = combined.pivot(index='User-ID', columns='Book-ID', values='Rating')
user_book_matrix.fillna(0, inplace=True)
users = user_book_matrix.index.tolist()
books = user_book_matrix.columns.tolist()
user_book_matrix = user_book_matrix
user_book_matrix_csr = csr_matrix(user_book_matrix)

book_user_matrix = user_book_matrix.T

book_user_matrix.head(10)

User-ID,A100YDQ9DIP06P,A102Z3T7NSM5KC,A106016KSI0YQ,A10A1S5NAQBT21,A10BZSGALQPS0V,A10F2RGD2D36ZI,A10G4BPT5MGBHY,A10IKHRUSMKP46,A10K3DLOEVMKW3,A10LWBOIZCF2QT,...,AYYIRXRLL0FZF,AYZN94ZQWIA4F,AZ216MK9KO1S0,AZ85B5Q1UEH5U,AZA5KI0FAD29V,AZFX7FLEHCWXM,AZHOZFC4VWIMF,AZK12FBX7X7LN,AZM9G2E1UMKRZ,AZPMV1PM3ISSL
Book-ID,,,,,,,,,,,,,,,,,,,,,
000100039X,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0001046314,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0001047868,1.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0001048767,0.0,0.0,0.0,1.0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
000105001X,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0001050230,0.0,0.0,0.0,1.0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0001052292,0.0,0.0,0.0,0.0,0.75,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0001381733,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0001384198,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
book_user_matrix.shape

(3233, 1913)

So now, we have transformed the tabular format to a pivot table, with Book-ID as rows, User-ID as columns, and the input values as the ratings. This table is fairly sparse.

In [7]:
from sklearn.neighbors import NearestNeighbors

model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(book_user_matrix)



NearestNeighbors(algorithm='brute', metric='cosine')

Model is created, we are using cosine similarity as the distance. It fit really fast.

In [8]:
A = model_knn.kneighbors_graph(book_user_matrix)
A = A.toarray()
query_index = np.random.choice(book_user_matrix.shape[0])
distances, indices = model_knn.kneighbors(np.array(book_user_matrix.iloc[query_index,:]).reshape(1,-1), n_neighbors = 6)


C:\Users\kenhu\anaconda3\envs\torch1\lib\site-packages\sklearn\base.py:441: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


In [9]:
distances

array([[1.11022302e-16, 2.40369836e-01, 4.49500147e-01, 4.84642298e-01,
        5.45497228e-01, 5.51660262e-01]])

In [10]:
A

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [11]:
for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(book_user_matrix.index[query_index]))
    else:
        print('{0}:{1}, with distance of {2}:'.format(i,book_user_matrix.index[indices.flatten()[i]], distances.flatten()[i]))

rng = model_knn.radius_neighbors(np.array(book_user_matrix.iloc[query_index,:]).reshape(1,-1), radius = 0.9, sort_results = True)
rng_0 = rng[0][0]


Recommendations for 0061242411:

1:0061355445, with distance of 0.24036983637516762:
2:0061341592, with distance of 0.4495001465839097:
3:0061355577, with distance of 0.4846422976531648:
4:0061344788, with distance of 0.5454972277828751:
5:006134477X, with distance of 0.5516602624600999:


C:\Users\kenhu\anaconda3\envs\torch1\lib\site-packages\sklearn\base.py:441: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


For book with asin number 0006497071, the closest five neighbors are listed. So we are clusterin based on cosine similarity

In [12]:
from scipy.sparse import csr_matrix

In [13]:
book_user_sparse = csr_matrix(book_user_matrix)

In [14]:
book_user_sparse

<3233x1913 sparse matrix of type '<class 'numpy.float64'>'
	with 52553 stored elements in Compressed Sparse Row format>

This thing helps compress data in sparse matrix

Gonna load in some book names so we can match ID to book title to have a better sense oof how the model is performing

In [15]:
##Get book names from the other product metadata json file

meta_json_path = 'meta_Books.json'

size = 100000
meta = pd.read_json(meta_json_path, lines=True,
                      chunksize=size)


In [16]:
book_ids = pd.DataFrame(combined_final['Book-ID'].unique())

In [17]:
book_ids = book_ids.rename(columns = {0: 'book-ID'})

book_ids

,book-ID
0,0001047868
1,0060219106
2,0060232943
3,0060766212
4,0060783338
...,...
3228,006169696X
3229,006012394X
3230,0025473905
3231,0007456107


In [18]:
book_list = []
for chunk_meta in meta:
    book_merged = book_ids.merge(chunk_meta[['asin','title']], how = 'inner' ,left_on ='book-ID', right_on = 'asin')
    book_list.append(book_merged)

In [19]:
len(book_list)

30

In [20]:
book_merged = pd.concat(book_list, ignore_index=True, join='outer', axis=0)

book_merged

,book-ID,asin,title
0,0001047868,0001047868,Kidnapped (HarperCollinsAudioBooks)
1,0060219106,0060219106,Harriet the Spy
2,0060232943,0060232943,Just so stories
3,0060766212,0060766212,Curse of the Bane (Last Apprentice)
4,0060783338,0060783338,The Dangerous Alphabet
...,...,...,...
3228,006169696X,006169696X,The Soulmate Secret: Manifest the Love of Your...
3229,006012394X,006012394X,Where Do We Go From Here: Choas or Community?
3230,0025473905,0025473905,Respect for Acting
3231,0007456107,0007456107,Collins World Atlas: Complete Edition


In [21]:
book_merged.dtypes

book-ID    object
asin       object
title      object
dtype: object

In [22]:
var = book_merged.loc[book_merged['asin'] == '0001047868', 'title'][0]
print(str(var))

Kidnapped (HarperCollinsAudioBooks)


In [23]:
query_index = np.random.choice(book_user_matrix.shape[0])
distances, indices = model_knn.kneighbors(np.array(book_user_matrix.iloc[query_index,:]).reshape(1,-1), n_neighbors = 6)

query_index

C:\Users\kenhu\anaconda3\envs\torch1\lib\site-packages\sklearn\base.py:441: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


795

In [24]:
for i in range(0, len(distances.flatten())):
    if i == 0:
        orig_asin = book_user_matrix.index[query_index]
        orig_name = book_merged.loc[book_merged['asin'] == orig_asin, 'title']

        print('Recommendations for {0}:\n'.format(orig_name))
    else:
        rec_asin = book_user_matrix.index[indices.flatten()[i]]
        rec_name = book_merged.loc[book_merged['asin'] == rec_asin, 'title']
        print('{0}:{1}, with distance of {2}:'.format(i,rec_name, distances.flatten()[i]))


Recommendations for 2083    A Dark So Deadly
Name: title, dtype: object:

1:2982    In the Cold Dark Ground (Logan McRae, Book 10)
Name: title, dtype: object, with distance of 0.4314647563850389:
2:2207    The Missing and the Dead (Logan McRae, Book 9)
Name: title, dtype: object, with distance of 0.4893722091966197:
3:2088    Saving Sophie: A Compulsively Twisty Psycholog...
Name: title, dtype: object, with distance of 0.6011379823912671:
4:2274    Just Rewards
Name: title, dtype: object, with distance of 0.6783878991170521:
5:1017    The Sandman
Name: title, dtype: object, with distance of 0.6919514761291159:


Seems to work decently, makes decent sense, but as with kNN, it is susceptible to certain effects. Also, need to remove those indices. Anyway, lets try out Matrix Factorization and see the results from that. Also, how do users fit into this? I am only visualizing book similarity, but have not incorporated how users come into play.

## Matrix Factorization

In [25]:
import sklearn
from sklearn.decomposition import TruncatedSVD
SVD = TruncatedSVD(n_components = 12, random_state = 17)
matrix = SVD.fit_transform(book_user_matrix)
matrix.shape

(3233, 12)

In [26]:
matrix_df = pd.DataFrame(matrix)
matrix_df

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.870501,0.604023,0.022925,-0.463339,-0.417020,0.148489,-0.086902,0.356707,-0.334824,-0.538694,-0.010489,-0.331856
1,0.146753,-0.030718,-0.104415,0.279727,-0.005239,-0.187486,0.068513,0.090569,0.056080,0.198894,-0.104109,0.035889
2,0.629724,0.352882,0.161631,0.384961,-0.160385,0.205592,-0.095776,-0.149553,-0.275102,-0.085819,0.227238,0.179629
3,4.379748,2.196811,1.346440,0.059411,-0.755889,1.274595,-0.856409,-0.869941,-1.215897,-1.255050,-0.551168,-0.030086
4,0.088915,-0.005958,0.003853,0.097692,0.028997,-0.033006,-0.084781,-0.055796,-0.017877,-0.009815,0.006942,0.091490
...,...,...,...,...,...,...,...,...,...,...,...,...
3228,0.140727,-0.016382,-0.128144,0.022384,-0.001315,-0.010358,0.080449,0.125698,0.045888,0.079762,0.167140,-0.135171
3229,0.094020,0.036538,-0.034875,-0.067781,-0.036424,0.001893,0.022655,0.110304,0.032577,-0.074109,0.168068,-0.134371
3230,0.074401,-0.001550,-0.065633,-0.057937,-0.022706,0.009561,0.045301,0.163310,0.028847,-0.043872,0.212119,-0.116486
3231,0.251671,-0.340068,-0.559796,0.014874,0.032601,0.174017,0.008363,-0.411406,-0.214260,0.078903,0.015280,-0.052992


In [27]:
import warnings
warnings.filterwarnings('ignore', category = RuntimeWarning)
corr = np.corrcoef(matrix)
corr.shape

(3233, 3233)

In [28]:
corr_df = pd.DataFrame(corr)

corr_df

,0,1,2,3,4,5,6,7,8,9,...,3223,3224,3225,3226,3227,3228,3229,3230,3231,3232
0,1.000000,-0.231647,0.556084,0.790007,-0.064338,0.743313,0.480669,0.587142,0.699004,0.382463,...,0.425761,0.017353,-0.001108,0.074128,0.032675,0.267583,0.618728,0.446867,-0.091178,0.216613
1,-0.231647,1.000000,0.034908,-0.127903,0.357600,-0.182360,-0.123170,0.212031,0.093621,0.018708,...,0.127277,0.404180,0.176362,0.366962,-0.011093,0.295312,-0.242826,-0.170037,0.214523,0.350419
2,0.556084,0.034908,1.000000,0.842308,0.622869,0.791453,0.664770,0.510415,0.408491,0.207742,...,0.334757,0.239953,0.350191,0.336821,0.158009,-0.020615,0.100697,-0.044080,0.281795,0.287688
3,0.790007,-0.127903,0.842308,1.000000,0.401060,0.984812,0.643376,0.516088,0.549485,0.203010,...,0.246001,0.157987,0.249891,0.233673,0.182735,-0.077749,0.176267,-0.047376,0.134171,0.153295
4,-0.064338,0.357600,0.622869,0.401060,1.000000,0.371031,0.479213,0.099957,-0.127619,-0.113930,...,0.129268,0.281280,0.451310,0.389913,0.539645,-0.249006,-0.359028,-0.401633,0.308346,0.142018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3228,0.267583,0.295312,-0.020615,-0.077749,-0.249006,-0.158786,0.110056,0.328443,0.333532,0.125616,...,0.704021,0.504653,0.371861,0.543918,-0.052031,1.000000,0.758945,0.828639,0.372705,0.759659
3229,0.618728,-0.242826,0.100697,0.176267,-0.359028,0.101885,0.208662,0.283932,0.369566,0.151250,...,0.633577,0.047011,0.062460,0.151994,0.073736,0.758945,1.000000,0.956792,-0.040915,0.388024
3230,0.446867,-0.170037,-0.044080,-0.047376,-0.401633,-0.135188,0.159064,0.242363,0.292520,0.198920,...,0.638500,0.111076,0.072890,0.187458,0.017029,0.828639,0.956792,1.000000,0.030512,0.473596
3231,-0.091178,0.214523,0.281795,0.134171,0.308346,0.108993,0.371103,0.088856,0.028935,-0.029973,...,0.510828,0.944717,0.934403,0.914085,-0.103643,0.372705,-0.040915,0.030512,1.000000,0.816097


In [29]:
query_index = np.random.choice(book_user_matrix.shape[0])
query_index

3216

In [30]:
corr_select_book = pd.DataFrame(corr[query_index])
corr_select_book.dropna(inplace = True)
corr_select_book.sort_values(0, ascending = False, inplace = True)

top_ten = corr_select_book.iloc[0:10,:]

top_ten.reset_index(inplace = True)

top_ten = top_ten.rename(columns = {'index':'Ind',0:'Distance'})

top_ten

,Ind,Distance
0,3216,1.000000
1,666,0.984373
2,934,0.980055
3,867,0.979402
4,2244,0.976788
5,2697,0.971687
6,643,0.968650
7,861,0.966488
8,2855,0.966414
9,703,0.966158


In [31]:
top_ten.loc[0,'Ind']

3216

In [32]:
for i in range(0,10):
    if i == 0:
        orig_asin = book_user_matrix.index[query_index]
        orig_name = book_merged.loc[book_merged['asin'] == orig_asin, 'title']

        print('Recommendations for {0}:\n'.format(orig_name))
    else:
        rec_asin = book_user_matrix.index[top_ten.loc[i,'Ind']]
        rec_name = book_merged.loc[book_merged['asin'] == rec_asin, 'title']
        print('{0}:{1}, with distance of {2}:'.format(i,rec_name, top_ten.loc[i,'Distance']))


Recommendations for 562    Let Me Be Frank With You: A Frank Bascombe Book
Name: title, dtype: object:

1:2656    My Dear I Wanted to Tell You
Name: title, dtype: object, with distance of 0.9843732677137663:
2:2557    The Portable Veblen: Shortlisted for the Baile...
Name: title, dtype: object, with distance of 0.9800551236732068:
3:1970    Barkskins: Longlisted for the Baileys Women's ...
Name: title, dtype: object, with distance of 0.9794019135184439:
4:2985    The Berlin Wall: A World Divided, 1961-1989
Name: title, dtype: object, with distance of 0.9767879493120055:
5:1612    Life Sentences: A Novel
Name: title, dtype: object, with distance of 0.971686593389236:
6:381    Woodcutter
Name: title, dtype: object, with distance of 0.9686498402653865:
7:2741    The Girl Who Saved the King of Sweden
Name: title, dtype: object, with distance of 0.9664880712240191:
8:1436    The Maytrees: A Novel
Name: title, dtype: object, with distance of 0.9664136559259902:
9:2000    The Core (The Demon 

Just trying out NMF (Non Negative Matrix Factorization) to see how it works

In [33]:
from sklearn.decomposition import NMF
NMF_model = NMF(n_components = 12, random_state = 17)
nn_matrix = NMF_model.fit_transform(book_user_matrix)
nn_matrix.shape

C:\Users\kenhu\anaconda3\envs\torch1\lib\site-packages\sklearn\decomposition\_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(


(3233, 12)

In [34]:
nn_matrix_df = pd.DataFrame(nn_matrix)
nn_matrix_df

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.115422,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.056684,0.237644,0.000000,0.000644,0.000000
1,0.000000,0.000000,0.006787,0.063655,0.000000,0.058985,0.000000,0.000000,0.000000,0.000000,0.007591,0.000000
2,0.045789,0.000000,0.000000,0.100136,0.081631,0.000000,0.000000,0.000000,0.016325,0.010543,0.000000,0.000000
3,0.536043,0.002156,0.000000,0.142429,0.325281,0.000000,0.000000,0.000000,0.202558,0.060639,0.000000,0.078045
4,0.003130,0.000000,0.001204,0.019512,0.000000,0.005165,0.000681,0.000000,0.000000,0.002437,0.000000,0.026904
...,...,...,...,...,...,...,...,...,...,...,...,...
3228,0.000000,0.000000,0.016098,0.009357,0.002760,0.003874,0.000000,0.061328,0.000000,0.000000,0.000000,0.000000
3229,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.061540,0.001222,0.000000,0.000000,0.002992
3230,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.068063,0.001031,0.000000,0.000000,0.000000
3231,0.000000,0.000000,0.190712,0.000000,0.000000,0.010813,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


More sparse than the other one for sure, but I guess more interpretable

Seems like any kind of learning built upon this requires some fitting of the embeddings, which is minimizing the errors of the predicted ratings and the theoretical ratings, using some sort of iterative gradient approach

### Breaking down SVD

In [134]:
import sklearn
from sklearn.decomposition import TruncatedSVD
SVD = TruncatedSVD(n_components = 256, random_state = 17)
book_embed = SVD.fit_transform(book_user_matrix)
book_embed.shape

SVD2 = TruncatedSVD(n_components = 256, random_state = 17)
user_embed = SVD2.fit_transform(user_book_matrix)
user_embed.shape


(1913, 256)

In [135]:
book_embed

array([[ 0.87050025,  0.60405784,  0.02296856, ..., -0.1285676 ,
         0.27229285,  0.23450857],
       [ 0.14675334, -0.03067907, -0.10434265, ...,  0.00507571,
         0.06632471,  0.00944298],
       [ 0.62972447,  0.35280326,  0.16148072, ...,  0.07038353,
        -0.11160038, -0.24763683],
       ...,
       [ 0.07440067, -0.00153492, -0.06565949, ...,  0.00128273,
         0.02638655, -0.05157164],
       [ 0.25167114, -0.34009581, -0.55984184, ..., -0.01120925,
         0.03843102,  0.05831682],
       [ 0.50735171, -0.36755862, -0.81910783, ...,  0.03703202,
        -0.28533834, -0.25256207]])

In [136]:
user_embed


array([[ 0.06022063,  0.00490159,  0.01727474, ..., -0.04912655,
         0.05583313,  0.03943066],
       [ 0.24274489,  0.18322569,  0.22203015, ...,  0.09040022,
         0.0253958 , -0.06454224],
       [ 0.16746397,  0.0459211 ,  0.07727452, ..., -0.07347334,
         0.09081606,  0.05407627],
       ...,
       [ 0.32463363,  0.26606466,  0.25234303, ..., -0.00784642,
         0.12640824,  0.09429858],
       [ 0.35025612,  0.09914433, -0.13433141, ..., -0.14663009,
        -0.03202544,  0.1556988 ],
       [ 1.03279154, -0.23477025, -0.31357028, ...,  0.06771256,
        -0.16019425,  0.03954872]])

In [137]:
user_book_matrix

Book-ID,000100039X,0001046314,0001047868,0001048767,000105001X,0001050230,0001052292,0001381733,0001384198,0001712713,...,0061703796,0061706515,006170654X,0061706558,0061706620,0061708712,0061708771,0061709697,0061710245,0061713244
User-ID,,,,,,,,,,,,,,,,,,,,,
A100YDQ9DIP06P,0.0,0.0,1.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A102Z3T7NSM5KC,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A106016KSI0YQ,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A10A1S5NAQBT21,0.0,0.0,0.0,1.0,0.0,1.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A10BZSGALQPS0V,0.0,0.0,0.0,0.0,0.0,0.0,0.75,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AZFX7FLEHCWXM,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0
AZHOZFC4VWIMF,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AZK12FBX7X7LN,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [138]:
book_embed_T = np.transpose(book_embed)

In [139]:
reconstruct_user = np.matmul(user_embed, book_embed_T)

reconstruct_user.shape

(1913, 3233)

In [140]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

reconstruct_user = scaler.fit_transform(reconstruct_user)

In [141]:
reconstruct = pd.DataFrame(reconstruct_user, index = user_book_matrix.index, columns = user_book_matrix.columns)

In [142]:
reconstruct

Book-ID,000100039X,0001046314,0001047868,0001048767,000105001X,0001050230,0001052292,0001381733,0001384198,0001712713,...,0061703796,0061706515,006170654X,0061706558,0061706620,0061708712,0061708771,0061709697,0061710245,0061713244
User-ID,,,,,,,,,,,,,,,,,,,,,
A100YDQ9DIP06P,0.125040,0.414768,0.112318,0.062552,0.317173,0.098614,0.347325,0.284334,0.157287,0.311408,...,0.451377,0.659308,0.680193,0.621408,0.461311,0.506837,0.450500,0.470776,0.748191,0.640582
A102Z3T7NSM5KC,0.124038,0.394634,0.134715,0.095284,0.284804,0.119613,0.312930,0.273279,0.166683,0.323144,...,0.471952,0.638682,0.676728,0.602741,0.428654,0.518160,0.454965,0.527916,0.739599,0.655271
A106016KSI0YQ,0.103301,0.429590,0.094497,0.066626,0.388295,0.115265,0.366436,0.276667,0.169800,0.303782,...,0.456612,0.654632,0.670147,0.611187,0.499232,0.537519,0.440786,0.486880,0.732350,0.655440
A10A1S5NAQBT21,0.223188,0.541699,0.143301,0.062708,0.462220,0.089198,0.309013,0.244665,0.173651,0.350244,...,0.515815,0.704618,0.738313,0.664952,0.408238,0.623618,0.683616,0.669680,0.799757,0.745116
A10BZSGALQPS0V,0.144966,0.338034,0.165876,0.084633,0.334000,0.141435,0.368145,0.283163,0.197730,0.247291,...,0.464362,0.661987,0.693145,0.604985,0.370419,0.465677,0.329313,0.397012,0.747861,0.645960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AZFX7FLEHCWXM,0.153459,0.452467,0.111998,0.047528,0.255191,0.092618,0.315509,0.250322,0.135964,0.309085,...,0.432905,0.639659,0.687656,0.552033,0.506671,0.405523,0.433524,0.465662,0.748070,0.655201
AZHOZFC4VWIMF,0.449278,0.465017,0.220188,0.227628,0.257089,0.242801,0.307136,0.387864,0.244841,0.370579,...,0.599939,0.708298,0.713220,0.670022,0.303464,0.658834,0.681684,0.721704,0.762145,0.787973
AZK12FBX7X7LN,0.186554,0.411315,0.110157,0.081799,0.324055,0.124947,0.294004,0.255903,0.133617,0.253250,...,0.428244,0.634859,0.645751,0.592114,0.582720,0.527491,0.421862,0.471861,0.712700,0.631158


In [143]:
from sklearn.metrics import mean_squared_error

In [144]:
score = mean_squared_error(user_book_matrix, reconstruct)

In [145]:
score

0.2005572892479286

Let's try Scipys SVD

In [195]:
from scipy.sparse.linalg import svds

U, Sigma, VT = svds(user_book_matrix, k = 256)

In [196]:
U.shape


(1913, 256)

In [197]:
pd.DataFrame(U)



,0,1,2,3,4,5,6,7,8,9,...,246,247,248,249,250,251,252,253,254,255
0,0.006689,-0.011366,-0.002294,0.000529,-0.003882,-0.009726,0.010845,-0.015494,-0.003110,-0.018914,...,0.001514,-0.004763,-0.002206,-0.000518,0.000758,0.003122,0.001396,0.000742,0.000208,0.001641
1,0.008263,-0.048720,-0.006581,0.000910,-0.021901,-0.012168,-0.000421,-0.013265,0.011323,-0.001383,...,0.003354,0.001222,-0.000999,-0.001505,0.004745,-0.001744,-0.002738,0.009536,0.007764,0.006614
2,0.008852,0.003281,-0.003875,-0.014015,-0.001939,-0.008805,-0.001164,-0.020106,-0.015613,-0.001512,...,0.003862,0.001688,0.001263,-0.001292,-0.002739,0.000637,-0.001630,0.003319,0.001946,0.004563
3,0.012523,-0.005813,0.015522,0.003384,-0.015882,0.020506,-0.007426,-0.019446,0.010144,0.003653,...,-0.007851,-0.056793,-0.041285,0.027649,0.030671,-0.005244,0.013762,-0.023397,-0.033354,0.031284
4,0.018243,0.005334,-0.000830,0.038230,-0.018765,0.006513,0.024551,0.013102,0.033718,0.030529,...,0.007078,0.008316,0.005351,0.001169,0.008373,-0.003057,-0.011827,-0.004820,0.011179,0.016474
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1908,0.001799,-0.000420,0.010180,0.004779,0.013150,0.012896,0.025496,0.023291,-0.002735,0.018842,...,-0.004164,0.003233,0.009111,-0.001796,-0.012058,0.002509,0.000574,0.004595,0.000054,0.004918
1909,0.022299,0.009019,-0.013194,0.004471,-0.004388,0.006739,-0.002746,0.003903,0.002602,-0.012931,...,-0.028563,-0.055346,-0.024554,-0.014943,-0.047591,-0.102362,-0.031331,-0.108397,0.147562,0.026918
1910,0.003041,-0.005683,0.017034,0.011514,-0.008751,0.012704,0.008624,-0.000133,-0.015244,-0.006806,...,0.012305,0.002211,-0.018669,0.011781,0.006265,0.009432,0.002298,0.010838,0.011274,0.008846
1911,-0.009549,-0.009737,0.006034,0.015230,0.023870,-0.021134,0.009547,-0.003970,0.030173,0.052692,...,0.013996,0.020535,0.004717,-0.003915,-0.003006,-0.018469,0.073367,-0.005770,0.004201,0.009544


In [198]:
Sigma = np.diag(Sigma)

Sigma.shape

(256, 256)

In [199]:
full = np.matmul(U, Sigma)

full.shape

(1913, 256)

In [205]:
full2 = np.matmul(full, VT)

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

full2 = scaler.fit_transform(full2)

In [206]:
scipy_svd = pd.DataFrame(full2, index = user_book_matrix.index, columns = user_book_matrix.columns)

scipy_svd

Book-ID,000100039X,0001046314,0001047868,0001048767,000105001X,0001050230,0001052292,0001381733,0001384198,0001712713,...,0061703796,0061706515,006170654X,0061706558,0061706620,0061708712,0061708771,0061709697,0061710245,0061713244
User-ID,,,,,,,,,,,,,,,,,,,,,
A100YDQ9DIP06P,0.184458,0.108782,0.202165,0.126966,0.226000,0.142588,0.078085,0.143481,0.085802,0.106975,...,0.148441,0.150123,0.172910,0.128578,0.177897,0.124630,0.087376,0.144104,0.059623,0.128377
A102Z3T7NSM5KC,0.190631,0.125778,0.103246,0.095717,0.224631,0.155414,0.107855,0.132239,0.090329,0.127638,...,0.123900,0.155865,0.123429,0.109481,0.176547,0.150884,0.102895,0.154343,0.097371,0.075966
A106016KSI0YQ,0.179201,0.117549,0.124577,0.164175,0.231647,0.119847,0.069026,0.124145,0.097793,0.114064,...,0.149030,0.164693,0.166159,0.124575,0.182909,0.149728,0.072527,0.181340,0.055473,0.118494
A10A1S5NAQBT21,0.226254,0.133251,0.148587,0.967033,0.259440,0.713395,0.094134,0.134500,0.071467,0.085927,...,0.123374,0.159894,0.121329,0.120678,0.224697,0.141170,0.111810,0.171499,0.061046,0.078218
A10BZSGALQPS0V,0.137775,0.127474,0.068181,0.100458,0.355286,0.221215,0.230653,0.135540,0.118353,0.130085,...,0.176619,0.155915,0.193477,0.115650,0.199401,0.144998,0.148432,0.123459,0.050554,0.088761
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AZFX7FLEHCWXM,0.168431,0.148274,0.132427,0.113086,0.314463,0.171385,0.103621,0.129644,0.070999,0.171306,...,0.170992,0.186492,0.122006,0.140697,0.210436,0.221942,0.141586,0.147498,0.034336,0.126927
AZHOZFC4VWIMF,0.146401,0.128280,0.090607,0.134099,0.172239,0.114085,0.087314,0.139936,0.090475,0.129591,...,0.132903,0.168879,0.142648,0.124809,0.214588,0.126908,0.097473,0.133265,0.058174,0.101700
AZK12FBX7X7LN,0.166364,0.121306,0.130185,0.145551,0.302619,0.096790,0.105452,0.119927,0.088729,0.113569,...,0.135092,0.173293,0.152473,0.079697,0.156083,0.125466,0.102859,0.155380,0.081098,0.110887


In [207]:
score = mean_squared_error(user_book_matrix, scipy_svd)

score

0.023934147500667642

In [208]:
full2.max()

1.0000000000000002

In [209]:
predicted_df = combined.copy()

predicted_df

,User-ID,Book-ID,Rating
0,A100YDQ9DIP06P,0001047868,1.00
1,A100YDQ9DIP06P,0060219106,1.00
2,A100YDQ9DIP06P,0060232943,0.75
3,A100YDQ9DIP06P,0060766212,1.00
4,A100YDQ9DIP06P,0060783338,0.75
...,...,...,...
53537,AZPMV1PM3ISSL,0060566264,0.75
53538,AZPMV1PM3ISSL,0060579528,0.75
53539,AZPMV1PM3ISSL,0060740221,0.75
53540,AZPMV1PM3ISSL,0061558257,0.50


In [210]:
def find_pivot_value(x):
    val = scipy_svd.loc[x[0],x[1]]
    return val

In [211]:
predicted_df['Pred_Rating'] = predicted_df[['User-ID','Book-ID']].apply(find_pivot_value, axis = 1)

predicted_df

,User-ID,Book-ID,Rating,Pred_Rating
0,A100YDQ9DIP06P,0001047868,1.00,0.202165
1,A100YDQ9DIP06P,0060219106,1.00,0.254680
2,A100YDQ9DIP06P,0060232943,0.75,0.143677
3,A100YDQ9DIP06P,0060766212,1.00,0.191757
4,A100YDQ9DIP06P,0060783338,0.75,0.181027
...,...,...,...,...
53537,AZPMV1PM3ISSL,0060566264,0.75,0.741983
53538,AZPMV1PM3ISSL,0060579528,0.75,0.587339
53539,AZPMV1PM3ISSL,0060740221,0.75,0.634009
53540,AZPMV1PM3ISSL,0061558257,0.50,0.388342


In [213]:
score = mean_squared_error(predicted_df['Rating'], predicted_df['Pred_Rating'])

score

0.10766170243660561